In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gc

In [ ]:
# TODO why difference between mask/norm before day shift and mask/norm after?
# NOTE
# 1. Number of nan points are similar
# 2. 

In [2]:
# # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# # # WITHOUT DAY SHIFT FIRST
# # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# # Load in raw regrid data
# fnam = "/home/jbassham/jack/thesis-rough/data/regrid/sh/20260107_164301/motion_ppv4_latlon_sh19922020_20260107_164301.npz"
# data = np.load(fnam, allow_pickle=True)

# ui = data['u'] # zonal ice velocity
# vi = data['v'] # meridional ice velocity
# ri = data['r'] # ice velocity uncertainty (same for u and v)

# print('Ice Velocity, Uncertainty Loaded')

# fnam = "/home/jbassham/jack/thesis-rough/data/regrid/sh/20260107_164301/con_nimbus7_latlon_sh19922020_20260107_164301.npz"
# data = np.load(fnam, allow_pickle=True)

# ci = data['ci']# ice concentration

# print('Concentration Loaded')

# fnam = "/home/jbassham/jack/thesis-rough/data/regrid/sh/20260107_164301/wind_jra55_latlon_sh19922020_20260107_164301.npz"
# data = np.load(fnam, allow_pickle=True)

# ua = data['u']
# va = data['v']

# print('Wind Loaded')

# # Delete 'data' from memory
# del data
# gc.collect()

# print('Variable Files Loaded')
# print('')

# # Mask ice concentration
# ci_raw = np.round(ci * 250) # raw value ice concentration (NSIDC)

# # NSIDC Masks 
# # 251 pole hole
# # 252 unused data
# # 253 coastline
# # 254 land
# ci = np.where((ci_raw == 251) | (ci_raw == 252) | (ci_raw == 253) | (ci_raw == 254), np.nan, ci)

# print('Raw concentration masked based on NSIDC masks.')

# # Create list of input variables
# invars = [ui, vi, ri, ua, va, ci]

# # Mask spatial indices with concentration less than .15, NaN concentration
# # NOTE keeping flag values for ice velocity uncertainties
# mask = (ci <= .15) | (np.isnan(ci))

# # NaN out points meeting mask condition
# invars_masked = [np.where(mask, np.nan, var) for var in invars]

# print('Inputs masked where ice concentration values <= .15 or nan.')
# print('')

# # NOTE: Normalization (z-score, for comparison between variables - 0 mean, 1 std)
# # 1. Compute temporal mean, gridwise
# # 2. Compute global standard deviation
# # 3. Remove mean and divde by standard deviation
# # 4. ** Ice velocities here are normalized by the standard deviation of the speed
# # 5. ** Uncertainty here is scaled by ci_std

# # Compute temporal mean of inputs at every gridpoint
# grid_means = [np.nanmean(var, axis = 0) for var in invars_masked]

# # Compute global stds
# global_stds = [np.nanstd(var) for var in invars_masked]

# # Unpacked masked variables
# ui_masked, vi_masked, ri_masked, ua_masked, va_masked, ci_masked = invars_masked

# # Delete unused arrays from memory
# del invars
# del ui, vi, ri, ua, va, ci
# gc.collect()

# # Unpack statistics
# ui_bar, vi_bar, _, ua_bar, va_bar, ci_bar = grid_means

# _, _, _, ua_std, va_std, ci_std = global_stds

# # Delete unused arrays from memory
# del _
# gc.collect()

# # Calculate speed
# Ui = np.sqrt(ui_masked ** 2 + vi_masked ** 2)

# # Get standard deviation of speed for normalization
# Ui_std = np.nanstd(Ui)

# # Delete unused arrays from memory
# del invars_masked, grid_means, global_stds
# gc.collect()

# # Normalize ice velocity and uncertainty by ice speed global standard deviation 
# # (z-score normalization)

# ui_norm = (ui_masked - ui_bar) / Ui_std
# vi_norm = (vi_masked - vi_bar) / Ui_std

# print("'uit_bar', 'vit_bar' normalized by 'cit_std:'")
# print(f"   {Ui_std:.3f} cm/s")
# print('')

# # Normalize uncertainty by standard deviation of speed
# ri_norm = ri_masked / Ui_std

# print(f"'rt' scaled by {Ui_std:.3f} cm/s:")
# print('')

# # Normalize remaning variables
# ua_norm = (ua_masked - ua_bar) / ua_std

# va_norm = (va_masked - va_bar) / va_std

# ci_norm = (ci_masked - ci_bar) / ci_std

# print("'ua', 'va', and 'ci' normalized by respective standard devations:")
# print(f"   {ua_std:.3f} cm/s, {va_std:.3f} cm/s, {ci_std:.3f}")
# print('')

# # Pack normalized input variables into list
# invars_norm = [ui_norm, vi_norm, ri_norm, ua_norm, va_norm, ci_norm]

# # Count number of data points in each variable
# total_points = [var.size for var in invars_norm]

# # Count the number of nans in each variable
# total_nan = [np.isnan(var).sum() for var in invars_norm]

# for p, n in zip(total_points, total_nan):
#     print(f"total points/ total nan: {p} / {n}")
#     print(f"num valid points {p - n}")
#     print(f"frac nan {p / n}")

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# WITH DAY SHIFT FIRST
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Load in raw regrid data
fnam = "/home/jbassham/jack/thesis-rough/data/regrid/sh/20260107_164301/motion_ppv4_latlon_sh19922020_20260107_164301.npz"
data = np.load(fnam, allow_pickle=True)

ui = data['u'] # zonal ice velocity
vi = data['v'] # meridional ice velocity
ri = data['r'] # ice velocity uncertainty (same for u and v)

print('Ice Velocity, Uncertainty Loaded')

fnam = "/home/jbassham/jack/thesis-rough/data/regrid/sh/20260107_164301/con_nimbus7_latlon_sh19922020_20260107_164301.npz"
data = np.load(fnam, allow_pickle=True)

ci = data['ci']# ice concentration

print('Concentration Loaded')

fnam = "/home/jbassham/jack/thesis-rough/data/regrid/sh/20260107_164301/wind_jra55_latlon_sh19922020_20260107_164301.npz"
data = np.load(fnam, allow_pickle=True)

ua = data['u']
va = data['v']

print('Wind Loaded')

# Delete 'data' from memory
del data
gc.collect()

print('Variable Files Loaded')
print('')

# Mask ice concentration
ci_raw = np.round(ci * 250) # raw value ice concentration (NSIDC)

# NSIDC Masks 
# 251 pole hole
# 252 unused data
# 253 coastline
# 254 land
ci = np.where((ci_raw == 251) | (ci_raw == 252) | (ci_raw == 253) | (ci_raw == 254), np.nan, ci)

print('Raw concentration masked based on NSIDC masks.')

# Shift present day parameters forward one day, for one point Middle Weddell
ui_t0 = ui[1:,:,:]
vi_t0 = vi[1:,:,:]
ua_t0 = ua[1:,:,:]
va_t0 = va[1:,:,:]
ri_t0 = ri[1:,:,:]

# Get present day ice concentration for masking
ci_t0 = ci[1:,:,:]

# Remove last day from previous day parameters
ci_t1 = ci[:-1,:,:]

# Create list of input variables
invars = [ui_t0, vi_t0, ri_t0, ua_t0, va_t0, ci_t1]

# Mask spatial indices with concentration less than .15, NaN concentration
# NOTE keeping flag values for ice velocity uncertainties
mask = (ci_t0 <= .15) | (np.isnan(ci_t0))

# NaN out points meeting mask condition
invars_masked = [np.where(mask, np.nan, var) for var in invars]

print('Inputs masked where ice concentration values <= .15 or nan.')
print('')

# NOTE: Normalization (z-score, for comparison between variables - 0 mean, 1 std)
# 1. Compute temporal mean, gridwise
# 2. Compute global standard deviation
# 3. Remove mean and divde by standard deviation
# 4. ** Ice velocities here are normalized by the standard deviation of the speed
# 5. ** Uncertainty here is scaled by ci_std

# Compute temporal mean of inputs at every gridpoint
grid_means = [np.nanmean(var, axis = 0) for var in invars_masked]

# Compute global stds
global_stds = [np.nanstd(var) for var in invars_masked]

# Unpacked masked variables
ui_masked, vi_masked, ri_masked, ua_masked, va_masked, ci_masked = invars_masked

# Delete unused arrays from memory
del invars
del ui, vi, ri, ua, va, ci
gc.collect()

# Unpack statistics
ui_bar, vi_bar, _, ua_bar, va_bar, ci_bar = grid_means

_, _, _, ua_std, va_std, ci_std = global_stds

# Delete unused arrays from memory
del _
gc.collect()

# Calculate speed
Ui = np.sqrt(ui_masked ** 2 + vi_masked ** 2)

# Get standard deviation of speed for normalization
Ui_std = np.nanstd(Ui)

# Delete unused arrays from memory
del invars_masked, grid_means, global_stds
gc.collect()

# Normalize ice velocity and uncertainty by ice speed global standard deviation 
# (z-score normalization)

ui_norm = (ui_masked - ui_bar) / Ui_std
vi_norm = (vi_masked - vi_bar) / Ui_std

print("'uit_bar', 'vit_bar' normalized by 'cit_std:'")
print(f"   {Ui_std:.3f} cm/s")
print('')

# Normalize uncertainty by standard deviation of speed
ri_norm = ri_masked / Ui_std

print(f"'rt' scaled by {Ui_std:.3f} cm/s:")
print('')

# Normalize remaning variables
ua_norm = (ua_masked - ua_bar) / ua_std

va_norm = (va_masked - va_bar) / va_std

ci_norm = (ci_masked - ci_bar) / ci_std

print("'ua', 'va', and 'ci' normalized by respective standard devations:")
print(f"   {ua_std:.3f} cm/s, {va_std:.3f} cm/s, {ci_std:.3f}")
print('')

# Pack normalized input variables into list
invars_norm = [ui_norm, vi_norm, ri_norm, ua_norm, va_norm, ci_norm]

# Count number of data points in each variable
total_points = [var.size for var in invars_norm]

# Count the number of nans in each variable
total_nan = [np.isnan(var).sum() for var in invars_norm]

for p, n in zip(total_points, total_nan):
    print(f"total points/ total nan: {p} / {n}")
    print(f"num valid points {p - n}")
    print(f"frac nan (invalid) {n / p}")

Ice Velocity, Uncertainty Loaded
Concentration Loaded
Wind Loaded
Variable Files Loaded

Raw concentration masked based on NSIDC masks.
Inputs masked where ice concentration values <= .15 or nan.



/tmp/ipykernel_114563/120357705.py:83: RuntimeWarning: Mean of empty slice
  grid_means = [np.nanmean(var, axis = 0) for var in invars_masked]


'uit_bar', 'vit_bar' normalized by 'cit_std:'
   7.937 cm/s

'rt' scaled by 7.937 cm/s:

'ua', 'va', and 'ci' normalized by respective standard devations:
   6.360 cm/s, 5.161 cm/s, 0.197

total points/ total nan: 447850944 / 305858109
num valid points 141992835
frac nan 1.4642441407365139
total points/ total nan: 447850944 / 305858109
num valid points 141992835
frac nan 1.4642441407365139
total points/ total nan: 447850944 / 305858109
num valid points 141992835
frac nan 1.4642441407365139
total points/ total nan: 447850944 / 293633495
num valid points 154217449
frac nan 1.5252038736248397
total points/ total nan: 447850944 / 293633495
num valid points 154217449
frac nan 1.5252038736248397
total points/ total nan: 447850944 / 293633495
num valid points 154217449
frac nan 1.5252038736248397


In [ ]:
# Define number of input channels
n_in = 3

# Define number of output channels
n_out = 2

# Get data dimensions
nt, nlat, nlon = np.shape(ui_t0) # time, latitude, longitude

# Initialize feature and target arrays (batch, channels, height, width)
x = np.zeros((nt, n_in, nlat, nlon)) # Features
y = np.zeros((nt, n_out, nlat, nlon)) # Targets

# Fill feature arrays
x[:, 0, :, :] = ua_norm # Zonal Wind, present day
x[:, 1, :, :] = va_norm # Meridional Wind, present day
x[:, 2, :, :] = ci_norm # Ice Concentration, previous day

# Fill target arrays
y[:, 0, :, :] = ui_norm # Zonal Ice Velocity, present day
y[:, 1, :, :] = vi_norm # Meridional Ice Velocity, present day

print("Feature and Target Arrays filled")

# Reshape uncertainty
# NOTE DO NOT reshape uncertainty for LR
# ri_t0 = np.expand_dims(ri_t0, 1)
# # ri_t0 = ri_t0.unsqueeze(1) # [nt, 1, nlat, nlon]

# Extract time (dates)
fnam = "/home/jbassham/jack/thesis-rough/data/coordinates/sh/20260107_164301/coord_sh19922020_20260107_164301.npz"
data = np.load(fnam, allow_pickle=True)
time = data['time']

# Create present day (t0) time coordinate variable by shifting forward one day
time_t0 = time[1:]

years = time_t0.astype('datetime64[Y]').astype(int) + 1970

# Define split mask based on years
train_mask = (years >= 1992) & (years <= 2016)
# val_mask   = (years >= 2017) & (years <= 2018)
test_mask  = (years >= 2019) & (years <= 2020)

# Get split indices
train_idx = np.where(train_mask)[0]
# val_idx   = np.where(val_mask)[0]
test_idx  = np.where(test_mask)[0]

# Fill train, validation, and test data arrays
x_train, y_train, r_train = x[train_idx], y[train_idx], ri_norm[train_idx]
# x_val, y_val, r_val = x[val_idx], y[val_idx], ri_t0[val_idx]
x_test, y_test, r_test = x[test_idx], y[test_idx], ri_norm[test_idx]

Feature and Target Arrays filled


In [13]:
# Save splits
fnam = 'new_exp_train_inputs.npz'

np.savez(
    fnam,
    x_train = x_train,
    y_train = y_train,
    r_train = r_train
)

# Save splits
fnam = 'new_exp_test_inputs.npz'

np.savez(
    fnam,
    x_test = x_test,
    y_test = y_test,
    r_test = r_test
)